In [2]:
import polars as pl
import matplotlib.pyplot as plt
import glob
import numpy as np
import math
import os
import glob

from tqdm import tqdm


In [3]:
#pl.read_parquet("/Users/kunkerdthaisong/Downloads/dataframe.parquet")

In [17]:
x=glob.glob("/Users/kunkerdthaisong/Downloads/SampleSkeleton/"+"*.npy")
sorted(x)

['/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A001.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A002.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A003.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A004.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A005.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A006.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A007.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A008.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A009.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R001A010.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001C001P001R002A001.skeleton.npy',
 '/Users/kunkerdthaisong/Downloads/SampleSkeleton/S001

In [12]:
df=pl.read_parquet("/Users/kunkerdthaisong/ipu/ntu_rgb_proj/spec/two_person_dataframe.parquet")
df

frame,joint,skel_body,x,y,z,dis_from_sensor,file_path,filename_n_class
i64,i64,i64,f64,f64,f64,f64,str,str
1,1,0,-0.851477,0.1916033,3.384463,1.064471,"""/Users/kunkerd…","""S001C001P001R0…"
1,2,0,-0.873564,0.4063588,3.341162,1.210748,"""/Users/kunkerd…","""S001C001P001R0…"
1,3,0,-0.893265,0.6162513,3.288516,1.370976,"""/Users/kunkerd…","""S001C001P001R0…"
1,4,0,-0.887127,0.7205115,3.303393,1.448603,"""/Users/kunkerd…","""S001C001P001R0…"
1,5,0,-0.904413,0.4904004,3.197891,1.283276,"""/Users/kunkerd…","""S001C001P001R0…"
1,6,0,-0.892437,0.2878125,3.192554,1.139078,"""/Users/kunkerd…","""S001C001P001R0…"
1,7,0,-0.830653,0.092949,3.252523,0.97768,"""/Users/kunkerd…","""S001C001P001R0…"
1,8,0,-0.823994,0.056782,3.269045,0.954492,"""/Users/kunkerd…","""S001C001P001R0…"
1,9,0,-0.851896,0.546665,3.398047,1.303607,"""/Users/kunkerd…","""S001C001P001R0…"


In [6]:
filename_n_class=[]
for i in df["file_path"]:
    filename_n_class.append(i.split("TO60/")[1])

In [9]:
df=df.with_columns(pl.Series(name="filename_n_class", values=filename_n_class)) 

In [11]:
df.write_parquet("/Users/kunkerdthaisong/ipu/ntu_rgb_proj/spec/two_person_dataframe.parquet")

In [140]:
# f_a=x.filter(pl.col("filename_n_class")=='S001C001P001R001A051.skeleton.npy',) #1m40s
#if it was sorted do linear search or your algo  #0.0s
for i,j in enumerate(x["filename_n_class"]):
    if j =="S001C001P002R002A050.skeleton.npy":
        break
print(i)

131500


In [138]:
f_30_a=x["filename_n_class"].unique(maintain_order=True)[:34]

In [149]:
x[:131499].write_parquet("/Users/kunkerdthaisong/ipu/ntu_rgb_proj/ntu_rgb/two_person_30actions_10class.parquet")

In [139]:
for i in f_30_a:
    print(i)

S001C001P001R001A050.skeleton.npy
S001C001P001R001A051.skeleton.npy
S001C001P001R001A052.skeleton.npy
S001C001P001R001A053.skeleton.npy
S001C001P001R001A054.skeleton.npy
S001C001P001R001A055.skeleton.npy
S001C001P001R001A056.skeleton.npy
S001C001P001R001A057.skeleton.npy
S001C001P001R001A058.skeleton.npy
S001C001P001R001A059.skeleton.npy
S001C001P001R001A060.skeleton.npy
S001C001P001R002A050.skeleton.npy
S001C001P001R002A051.skeleton.npy
S001C001P001R002A052.skeleton.npy
S001C001P001R002A053.skeleton.npy
S001C001P001R002A054.skeleton.npy
S001C001P001R002A055.skeleton.npy
S001C001P001R002A056.skeleton.npy
S001C001P001R002A057.skeleton.npy
S001C001P001R002A058.skeleton.npy
S001C001P001R002A059.skeleton.npy
S001C001P001R002A060.skeleton.npy
S001C001P002R001A050.skeleton.npy
S001C001P002R001A051.skeleton.npy
S001C001P002R001A052.skeleton.npy
S001C001P002R001A053.skeleton.npy
S001C001P002R001A054.skeleton.npy
S001C001P002R001A055.skeleton.npy
S001C001P002R001A056.skeleton.npy
S001C001P002R0

In [137]:
f_30_a[32]

'S001C001P002R001A060.skeleton.npy'

In [114]:
x["filename_n_class"][44750]

'S001C001P001R001A060.skeleton.npy'

In [6]:
glob.glob("/Users/kunkerdthaisong/ipu/ntu_rgb_proj/SampleSkeleton/*.npy")

[]

In [4]:
VIDEO_ = np.load("/Users/kunkerdthaisong/Downloads/skeletons20class_sampling/data/project/proj0173-action/Skeleton_Coordinate/raw_npy1TO60/", mmap_mode=None, allow_pickle=True)  

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kunkerdthaisong/Downloads/skeletons20class_sampling/data/project/proj0173-action/Skeleton_Coordinate/raw_npy1TO60/'